In [66]:
# WITH STRUCTURED OUTPUT and with classes for structured output
from langchain_groq import ChatGroq
from typing import TypedDict               ## Using Typedict first 
from dotenv import load_dotenv
import os
load_dotenv()
llm=ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=1.5,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)
# we dont need to write the prompt that you are and AI agent etc. etc., it is done by itself using TypedDict
class Review(TypedDict):
    summary:str
    sentiment:str

structured_llm=llm.with_structured_output(Review)

result=structured_llm.invoke(""" The hardware is great, but the software feels bloated.
There are roo many pre-installed apps that I can't remove
Also the UI is not looking good, can you fix it""")

print(result)

{'sentiment': 'Negative', 'summary': "The device's photo quality is excellent with the camera performing well in both daylight and night. The processor is the fastest I've ever used. However, UI needs optimization: apps like Facebook and Instagram are slow, there's freezing when switching apps and videos lag."}


In [67]:
# WITH STRUCTURED OUTPUT and with classes for structured output
from langchain_groq import ChatGroq
from typing import TypedDict,Annotated               ## Using Typedict first 
from dotenv import load_dotenv
load_dotenv()

llm=ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=1.5,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)
# Using Annotated, we can define what we exactly need, because we dont want our llm to hallucinate or give wrong outputs

class Review(TypedDict):
    summary:Annotated[str,"A brief summary of the review"]
    sentiment:Annotated[str,"Sentiment of the review, either good, bad or neutral"]

structured_llm=llm.with_structured_output(Review)

result=structured_llm.invoke(""" The hardware is great, but the software feels bloated.
There are roo many pre-installed apps that I can't remove
Also the UI is not looking good, can you fix it""")

print(result)

{'sentiment': 'bad', 'summary': 'The hardware is great, but the software feels bloated and lacks visual appeal. There are too many pre-installed apps that cannot be removed, which is inconvenient. The UI needs improvement.'}


In [68]:
# WITH STRUCTURED OUTPUT and with classes for structured output
from langchain_groq import ChatGroq
from typing import TypedDict,Annotated,Optional,Literal              ## Using Typedict first 
from dotenv import load_dotenv
load_dotenv()

llm=ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=1.5,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)
# Using Annotated, we can define what we exactly need, because we dont want our llm to hallucinate or give wrong outputs

class Review(TypedDict):
    key_themes:Annotated[list[str],"Write down all the key themes discussed in the review in a list"]
    summary:Annotated[str,"A brief summary of the review"]
    sentiment:Annotated[str,"Sentiment of the review, either good, bad or neutral"] # Both are correct, one returns the string and other returns the literal
    # sentiment:Annotated[Literal["pos","neg"],"Return the sentiment of the review"]
    pros:Annotated[Optional[list[str]], "Write down all the pros in the list"] # These are optional fields i.e. it can be present or absent
    cons:Annotated[Optional[list[str]], "Write down all the cons in the list"] # These are optional fields i.e. it can be present or absent
    name:Annotated[Optional[str],"Write down the name of the reviewer"] # Since name is not present in the paragraph so it will not display name

structured_llm=llm.with_structured_output(Review)

result=structured_llm.invoke(""" The hardware is great, but the software feels bloated.
There are roo many pre-installed apps that I can't remove
Also the UI is not looking good, can you fix it
Reviewed by SID """)

print(result)
print(result['sentiment'])
print(result['name'])

{'cons': ['The software feels bloated', 'Too many pre-installed apps', 'UI is not visually appealing', 'UI is difficult to navigate'], 'key_themes': ['Hardware quality', 'Software issues', 'UI/UX problems'], 'name': 'SID', 'pros': ['The hardware is great'], 'sentiment': 'neutral', 'summary': "The product's hardware is excellent, but the software has some issues."}
neutral
SID


In [69]:
### Pydantic demo
# ## Pydantic is a data validation and data parsing library for Python. It ensures that the data you work with is correct, structured and type-safe

from pydantic import BaseModel, EmailStr, Field
from typing import Optional
# class Student(BaseModel):
#     name:str

# new_student={'name':'sid'}

######## If you want to pass a default and Optional value. #########

class Student(BaseModel):
    name: str = 'SID'        # It is default now
    age: Optional[int] = None
    email: EmailStr
    cgpa: float=Field(gt=0,lt=10)
new_student={'age':30 , 'email':'abc@gmail.com','cgpa':5}      # Even if you pass the age in str and it is a valid number the pydantic is smart enough to convert it into int and so we do not get any err.

student= Student(**new_student)
print(student)
print(type(student))

student_dict=dict(student)
print(student_dict['age'])

student_json=student.model_dump_json()
print(student_json)

name='SID' age=30 email='abc@gmail.com' cgpa=5.0
<class '__main__.Student'>
30
{"name":"SID","age":30,"email":"abc@gmail.com","cgpa":5.0}


In [72]:
# WITH STRUCTURED OUTPUT and with classes for structured output
# now with Pydantic instead to Typedict
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
# from typing import TypedDict,Annotated,Optional,Literal              ## Using Typedict first 
from dotenv import load_dotenv
load_dotenv()

llm=ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=1.5,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)
# Using Annotated, we can define what we exactly need, because we dont want our llm to hallucinate or give wrong outputs

class Review(BaseModel):

    key_themes:list[str]=Field(description="Write down all the key themes discussed in the review in a list")
    summary:list[str]=Field(description="A brief summary of the review")
    sentiment:Literal["pos","neg","neutral"]=Field(description="Sentiment of the review, either good, bad or neutral")
    pros: Optional[list[str]]=Field(default=None, description="Write down all the pros in the list")
    cons: Optional[list[str]]=Field(default=None,description="Write down all the cons in the list")
    name: Optional[str]=Field(default=None,description="Write down the name of the reviewer")

structured_llm=llm.with_structured_output(Review)

result= structured_llm.invoke(""" The hardware is great, but the software feels bloated. There are roo many pre-installed apps that I can't remove Also the UI is not looking good, can you fix it Reviewed by SID """)

print(result)
print(result.name)

key_themes=['Software performance', 'Bloatware', 'Poor UI design'] summary=['The hardware is great but the software feels bloated and has too many pre-installed apps, plus the UI is not looking good.'] sentiment='neg' pros=['hardware is great'] cons=['software feels bloated', 'too many pre-installed apps', 'UI is not looking good'] name='SID'
SID


In [ ]:
###### NOW FOR JSON SCHEMA

: 

: 

In [ ]:
#### What is the difference between Structured output and Output parser
###  Structured output is an integrated form i.e. an ai model can have integrated structured output and we can use it so that the output is structured while if we use output parsers, whether the llm model supports the structured output or not, we can use it to get the structured output. We can also use the parsers to fetch info of one to other eg->

: 

: 

In [73]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
load_dotenv()

llm=ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)

template1= PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

template2= PromptTemplate(
    template='Write a short 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

# Either you can use this 

# prompt1=template1.invoke({'topic':'black hole'})
# result1=llm.invoke(prompt1)
# prompt2=template2.invoke({'text':result1.content})
# result2=llm.invoke(prompt2)
# print(result2.content)

# OR
parser=StrOutputParser()
chain = template1 | llm | parser | template2 | llm | parser
print(chain.invoke({'topic':'black hole'}))


Black holes are fascinating cosmic phenomena formed from the collapse of massive stars, characterized by intense gravity, an event horizon, and a singularity. They exist in various types, including stellar, supermassive, and rotating black holes, each with unique properties. Their effects on spacetime, such as frame-dragging and gravitational waves, are significant and detectable through electromagnetic radiation and advanced observatories. Beyond their scientific importance, black holes inspire cultural and philosophical thought, challenging our understanding of the universe. This report underscores their significance in advancing astrophysical knowledge and unraveling cosmic mysteries.


In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
load_dotenv()

llm=ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)

parser = JsonOutputParser()

template=PromptTemplate(
    template='Give me the name, age and city of a fictional person \n {format_instruction}',
    input_variables=[],
    partial_variables={'format_instruction':parser.get_format_instructions()},

)
# prompt =template.format()
# result= llm.invoke(prompt)
# print(result.content)
# final_result=parser.parse(result.content)
# print(final_result)
#  or you can use

chain = template | llm | parser
result=chain.invoke({}) # you have to send a blank dict
print(result)

{'name': 'Ethan Carter', 'age': 32, 'city': 'New York City'}


In [83]:
### NOW STRUCTURED OUTPUT PARSERS
#### USING THIS WE CAN ENFORCE OUR DESIRED SCHEMA TO THE LLM

from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
load_dotenv()

llm=ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)

schema= [
    ResponseSchema(name='fact_1',description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2',description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3',description='Fact 3 about the topic')
]
parser =StructuredOutputParser.from_response_schemas(schema)

template = PromptTemplate(
    template='Give 3 facts about {topic} \n {format}',
    input_variables=['topic'],
    partial_variables={'format':parser.get_format_instructions()}
)

# prompt=template.invoke({'topic':'black hole'})
# result= llm.invoke(prompt)
# final_result=parser.parse(result.content)
# print(final_result)



chain = template | llm | parser
result= chain.invoke({'topic':'Black hole'})
print(result)


{'fact_1': 'Black holes are regions in space where the gravitational pull is so strong that nothing, including light, can escape once it falls inside the event horizon.', 'fact_2': 'There are four types of black holes: stellar black holes, supermassive black holes, intermediate-mass black holes, and miniature black holes.', 'fact_3': 'Black holes are not visible, but their presence can be inferred by the effects they have on nearby stars, gas, and light, such as bending the light around them through gravitational lensing.'}


In [87]:
##### Pydantic Output Parser
### We can enforce schema as well as can validate the data which can't be done in structured output parsers

from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
load_dotenv()

llm=ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)

class Person(BaseModel):
    name:str=Field(description='Name of the person')
    age: int=Field(description='Age of the person')
    city:str=Field(description='Name of the city in which the person belongs')

parser = PydanticOutputParser(pydantic_object=Person)
template=PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format}',
    input_variables=['place'],
    partial_variables={'format':parser.get_format_instructions()}
)

# prompt=template.invoke({'place':'indian'})
# result=llm.invoke(prompt)
# final_result=parser.parse(result.content)
# print(final_result)

chain =template |llm|parser
print(chain.invoke({'place':'India'}))



name='Rahul Sharma' age=32 city='Bangalore'
